In [1]:
! $HOME/Documents/VirtualEnvs/keras/bin/activate && . /usr/local/bin/thisroot.sh

/bin/sh: /Users/adrianodiflorio/Documents/VirtualEnvs/keras/bin/activate: Permission denied


In [2]:
import ROOT
from ROOT import TFile,TH1,TH1F,TCanvas,TNtuple,TTreeReader,TTreeReaderValue
from ROOT import RooFit
from ROOT.RooFit import Layout
from ROOT import RooStats
from ROOT import RooAbsData
RooAbsData.setDefaultStorageType ( RooAbsData.Tree )
from array import array
import sys

Welcome to JupyROOT 6.10/08


In [3]:
from ROOT import RooRealVar,RooAbsPdf,RooChebychev,RooExponential,RooGaussian,RooAbsPdf,RooPlot,RooAddPdf,RooDataHist,RooArgSet,RooArgList
from ROOT import kGreen,kRed,kBlack,kBlue,kDashed,kDotted,kMagenta,RooVoigtian
from ROOT.RooFit import Components,LineColor,LineStyle,Name,Normalization,Range,SelectVars
from ROOT import RooDataSet,RooFormulaVar,RooLinkedList

In [4]:
rootfile = "/Users/adrianodiflorio/Desktop/X4140_roots/mumukk_tree.root" 
inputfile = TFile(rootfile,"READ") 
xTuple = (inputfile.Get("outuple")) 

In [5]:
massmin = 5.2
massmax = 5.55

In [6]:
myReader = TTreeReader("outuple", inputfile)
nentries = xTuple.GetEntries()
print nentries

9329742


In [7]:
massbins = (6.0 - 4.0)/0.005
mass = RooRealVar("xM","M(#mu#muKK)[GeV]",4.0,6.0)
mass.setBins(int(massbins))
lxy = RooRealVar("xL","l(xy)",0.0,10000.)
hlt = RooRealVar("xHlt","xHlt",0.0,20.0)
masskk = RooRealVar("kkM","kkM",0.5,1.5)
massbins = 200
masskk.setBins(int(massbins))
massmumu = RooRealVar("mumuM","mumuM",2.5,3.5)
cutFormula = RooFormulaVar("cutFormula","cutFormula","xHlt!=8.0",RooArgList(hlt))

In [8]:
alldata = RooDataSet("alldata","alldata",xTuple,RooArgSet(masskk,mass,lxy,hlt,massmumu))#,cutFormula)
datasetfile = TFile("xMassDataset.root","RECREATE") 
datasetfile.cd()
alldata.Write()

142810474

[#1] INFO:Eval -- RooAbsReal::attachToTree(kkM) TTree Float_t branch kkM will be converted to double precision
[#1] INFO:Eval -- RooAbsReal::attachToTree(xM) TTree Float_t branch xM will be converted to double precision
[#1] INFO:Eval -- RooAbsReal::attachToTree(xL) TTree Float_t branch xL will be converted to double precision
[#1] INFO:Eval -- RooAbsReal::attachToTree(xHlt) TTree Float_t branch xHlt will be converted to double precision
[#1] INFO:Eval -- RooAbsReal::attachToTree(mumuM) TTree Float_t branch mumuM will be converted to double precision
[#1] INFO:Eval -- RooTreeDataStore::loadValues(alldata) Ignored 27 out of range events


In [9]:
alldata.numEntries()

9329715

In [10]:
#xb->setRange("alt","x_coarse_bin1,x_coarse_bin3,x_coarse_bin5,x_coarse_bin7,x_coarse_bin9") ;
b0dataNonPrompt = ((alldata.reduce('xHlt!=8')).reduce('xM>5.2')).reduce("xLxt>3.0")

KeyboardInterrupt: 

[#0] ERROR:InputArguments -- RooFormula::RooFormula(xLxt>3.0): compile error
[#0] ERROR:InputArguments -- RooFormula::RooFormula(xLxt>3.0): compile error
[#0] ERROR:Eval -- RooFormula::eval(xLxt>3.0): Formula doesn't compile: xLxt>3.0
[#0] ERROR:Eval -- RooFormula::eval(xLxt>3.0): Formula doesn't compile: xLxt>3.0
[#0] ERROR:Eval -- RooFormula::eval(xLxt>3.0): Formula doesn't compile: xLxt>3.0
[#0] ERROR:Eval -- RooFormula::eval(xLxt>3.0): Formula doesn't compile: xLxt>3.0
[#0] ERROR:Eval -- RooFormula::eval(xLxt>3.0): Formula doesn't compile: xLxt>3.0
[#0] ERROR:Eval -- RooFormula::eval(xLxt>3.0): Formula doesn't compile: xLxt>3.0
[#0] ERROR:Eval -- RooFormula::eval(xLxt>3.0): Formula doesn't compile: xLxt>3.0
[#0] ERROR:Eval -- RooFormula::eval(xLxt>3.0): Formula doesn't compile: xLxt>3.0
[#0] ERROR:Eval -- RooFormula::eval(xLxt>3.0): Formula doesn't compile: xLxt>3.0
[#0] ERROR:Eval -- RooFormula::eval(xLxt>3.0): Formula doesn't compile: xLxt>3.0
[#0] ERROR:Eval -- RooFormula::eval(

Error in <RooFormula::Compile>:  Bad numerical expression : "xLxt"
Error in <RooFormula::Compile>:  Bad numerical expression : "xLxt"


In [ ]:
b0dataNonPromptMass = b0dataNonPrompt.reduce(SelectVars(RooArgSet(mass)))
b0dataNonPrompt.numEntries()

In [ ]:
c = TCanvas("canvas","canvas",1200,800) 
mass.setRange("fitRange",massmin,massmax)
mass.setBins(200)
massFrame = mass.frame(Range(massmin,massmax))
b0dataNonPrompt.plotOn(massFrame)
massFrame.Draw()
c.SaveAs("testmass.png")

In [ ]:
sigma1 = RooRealVar("sigma1","width of gaussian1",0.002,0.0005,0.05);
sigma2 = RooRealVar("sigma2","width of gaussian2",0.004,0.004,0.01);

phimean = 1.019

sigma = RooRealVar("sigma","width of gaussian",0.0013)
gamma = RooRealVar("gamma","gamma of bw",0.004253,0.001,0.01)
mean = RooRealVar("mean","mean of voigtian",phimean,phimean-0.005,phimean+0.005);

a0 = RooRealVar("a0","a0",0.001,-1.,1.)
a1 = RooRealVar("a1","a1",0.001,-0.5,0.5)
a2 = RooRealVar("a2","a2",-0.00001,-2.,2.)
a3 = RooRealVar("a3","a3",-0.000001,-0.1,0.1)
a4 = RooRealVar("a4","a4",-0.000001,-2.,2.)
a5 = RooRealVar("a5","a5",-0.000001)
a6 = RooRealVar("a6","a6",-0.000001,-0.01,0.01)

aset = RooArgList(a0,a1,a2)#,a3)

B_1     = RooRealVar ( "B_1"    , "B_1 "    , 0.3  , -20   , 100   )
B_2     = RooRealVar ( "B_2"    , "B_2"    , 0.3  , -20   , 100   )
B_3     = RooRealVar ( "B_3"    , "B_3"    , 0.3  , -20   , 100   )
B_4     = RooRealVar ( "B_4"    , "B_4"    , 0.3  , -20   , 100   )

gaussFrac = RooRealVar("sig1frac","fraction of component 1 in signal",0.3,0.0,1.0)
nSig = RooRealVar("nSig","nSig",100000,0.,10E6)
nBkg = RooRealVar("nBkg","nBkg",55000,0.,10E6)

In [14]:
signal = RooVoigtian("signal","signal",masskk,mean,gamma,sigma)
bkg    = RooBernstein("pdfB" , "pdfB"    , masskk   , RooArgList(B_1, B_2,B_3,B_4))
tot = RooAddPdf("tot","g+cheb",RooArgList(signal,bkg),RooArgList(nSig,nBkg))

mean.setValV(phimean)
gamma.SetConstant(ROOT.kTRUE)
mean.setConstant(ROOT.kTRUE)

In [ ]:
rfit = tot.fitTo(b0dataNonPrompt,Range(massmin,massmax))
mean.setConstant(ROOT.kFALSE)
rfit = tot.fitTo(b0dataNonPrompt,Range(massmin,massmax))
gamma.SetConstant(ROOT.kFALSE)
rfit = tot.fitTo(b0dataNonPrompt,Range(massmin,massmax))

In [15]:
rfit = tot.fitTo(b0dataNonPrompt,Range(massmin,massmax))

[#1] INFO:Minization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Eval -- RooRealVar::setRange(xM) new range named 'fit' created with bounds [5.2,5.55]
[#1] INFO:Fitting -- RooAbsOptTestStatistic::ctor(nll_tot_alldata) constructing test statistic for sub-range named fit
[#1] INFO:Eval -- RooRealVar::setRange(xM) new range named 'NormalizationRangeForfit' created with bounds [4,6]
[#1] INFO:Eval -- RooRealVar::setRange(xM) new range named 'fit_nll_tot_alldata' created with bounds [5.2,5.55]
[#1] INFO:Fitting -- RooAbsOptTestStatistic::ctor(nll_tot_alldata) fixing interpretation of coefficients of any RooAddPdf to full domain of observables 
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#0] WARNING:Optimization -- RooAbsOptTestStatistic::optimizeConstantTerms(nll_tot_alldata) WARNING Cache-and-track optimization (Optimize level 2) is only available for datasets implement in terms of RooVectorDataStore

In [16]:
massFrame = mass.frame(Range(massmin,massmax))
b0dataNonPrompt.plotOn(massFrame,RooLinkedList())
tot.plotOn(massFrame)

massFrame.Draw()
c.SaveAs("testmassFit.png")

[#1] INFO:Plotting -- RooAbsPdf::plotOn(tot) p.d.f was fitted in range and no explicit plot,norm range was specified, using fit range as default
[#1] INFO:Plotting -- RooAbsPdf::plotOn(tot) only plotting range 'fit_nll_tot_alldata'
[#1] INFO:Plotting -- RooAbsPdf::plotOn(tot) p.d.f. curve is normalized using explicit choice of ranges 'fit_nll_tot_alldata'


Info in <TCanvas::Print>: png file testmassFit.png has been created


In [17]:
cD=TCanvas("cD","cD",750,600);cD.cd()
splot   = RooStats.SPlot ( "sPlot","sPlot", b0dataNonPrompt, tot, RooArgList(nSig,nBkg))

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#0] WARNING:Optimization -- RooAbsOptTestStatistic::optimizeConstantTerms(nll_tot_alldata) WARNING Cache-and-track optimization (Optimize level 2) is only available for datasets implement in terms of RooVectorDataStore - ignoring this option for current dataset
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (model,cheb)
[#1] INFO:Fitting -- RooAbsPdf::fitTo(tot) Calculating sum-of-weights-squared correction matrix for covariance matrix
[#1] INFO:Minization -- RooMinimizer::optimizeConst: deactivating const optimization
[#1] INFO:InputArguments -- Printing Yields
RooArgList:: = (nSig,nBkg)
[#1] INFO:InputArguments -- yield in pdf: nSig 316885
[#1] INFO:InputArguments -- yield in pdf: nBkg 113254
[#0] ERROR:Eval -- RooAbsReal::logEvalError(cheb) evaluation error, 
 origin       : RooChebychev::cheb[ x=xM coefficients=(a0,a1,a2) ]
 

In [18]:
dstree  = b0dataNonPrompt.store().tree()
dstree.GetEntryNumber(88)

88L

In [19]:
shist   = TH1F('shist','shist', 100, 1.00, 1.05)

In [20]:
shist.Sumw2()
shist.SetLineColor(2)    
shist.SetMarkerColor(2); shist.SetMinimum(0.)
dstree.Project('shist','kkM','nSig_sw');  

In [21]:
shist.Draw('e0');
cD.SaveAs('OtherPlot.gif')

Info in <TCanvas::Print>: gif file OtherPlot.gif has been created


In [39]:
sys.exit()
xdataPrompt = (alldata.reduce('xM<4.8')).reduce('xM>4.0').reduce("xL<2.0")

In [11]:
massmin = 1.020-0.03
massmax = 1.020+0.03
phimean = 1.020
xdataPrompt.numEntries()

2983870

In [45]:
a0 = RooRealVar("a0","a0",0.001,-1.,1.)
a1 = RooRealVar("a1","a1",0.001,-0.5,0.5)
a2 = RooRealVar("a2","a2",-0.00001,-2.,2.)
a3 = RooRealVar("a3","a3",0.0)#
a4 = RooRealVar("a4","a4",0.0,-0.1,0.1)
a5 = RooRealVar("a5","a5",0.0,-0.025,0.05)
a6 = RooRealVar("a6","a6",0.0,-0.001,0.001)

aset = RooArgList(a0,a1,a2)#,a3,a4,a5)

sigma = RooRealVar("sigma","width of gaussian",0.0013)
gamma = RooRealVar("gamma","gamma of bw",0.004253)#,0.001,0.01)
mean = RooRealVar("mean","mean of gaussian",phimean,phimean-0.005,phimean+0.005);

nSig = RooRealVar("nSig","nSig",1E6,0.,5.0E6)
nBkg = RooRealVar("nBkg","nBkg",5E5,0.,5.0E6)
cheb = RooChebychev("cheb","Background",masskk,aset)
#gauss = RooGaussian("gauss","gaussian PDF ",mass,mean,sigma)
signal = RooVoigtian("signal","signal",masskk,mean,gamma,sigma)

tot = RooAddPdf("tot","g+cheb",RooArgList(signal,cheb),RooArgList(nSig,nBkg))

In [50]:
masskk.setBins(100)
mass.setBins(100)
h = xdataPrompt.createHistogram(masskk,mass,20,20)
h1 = h.ProjectionX()
dh1 = RooDataHist("kmass","kmass",RooArgList(masskk),h1)

In [46]:
rPhifit = tot.fitTo(xdataPrompt,Range(massmin,massmax))

[#1] INFO:Minization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Eval -- RooRealVar::setRange(kkM) new range named 'fit' created with bounds [0.99,1.05]
[#1] INFO:Fitting -- RooAbsOptTestStatistic::ctor(nll_tot_alldata) constructing test statistic for sub-range named fit
[#1] INFO:Eval -- RooRealVar::setRange(kkM) new range named 'NormalizationRangeForfit' created with bounds [0.5,1.5]
[#1] INFO:Fitting -- RooAbsOptTestStatistic::ctor(nll_tot_alldata) fixing interpretation of coefficients of any RooAddPdf to full domain of observables 
[#1] INFO:NumericIntegration -- RooRealIntegral::init(signal_Int[kkM]) using numeric integrator RooIntegrator1D to calculate Int(kkM)
[#1] INFO:NumericIntegration -- RooRealIntegral::init(signal_Int[kkM|NormalizationRangeForfit]_Norm[kkM]) using numeric integrator RooIntegrator1D to calculate Int(kkM)
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#0] WARNING:Optimiza

In [51]:
c = TCanvas("canvas","canvas",1200,800) 
phiFrame = masskk.frame(Range(massmin,massmax))
dh1.plotOn(phiFrame,RooLinkedList())
tot.plotOn(phiFrame)

phiFrame.Draw()
c.SaveAs("testmassFitPhi2.png")
c.Clear()